In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [998 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ge

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark import SparkFiles
from pyspark.sql.functions import avg, round, year

In [3]:
URL = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [4]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# 1. Read in the AWS S3 bucket into a DataFrame.
# 1.1. Download the file
spark.sparkContext.addFile(URL)
# 1.2. Determine the local path to the file
home_csv_path = f"file://{SparkFiles.get(os.path.basename(URL))}"
# 1.3. Load the local file into a dataframe
homes_df = spark.read.csv(home_csv_path, header=True, inferSchema=True)

homes_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
homes_df.createOrReplaceTempView('homes')

In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_4br_query = """
  SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS Average_Price
  FROM homes
  WHERE bedrooms=4
  GROUP BY Year
  ORDER BY Year DESC
"""
spark.sql(avg_4br_query).show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [7]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_3br_query = """
  SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS Average_Price
  FROM homes
  WHERE bedrooms=3
  AND bathrooms=3
  GROUP BY Year
  ORDER BY Year DESC
"""
spark.sql(avg_3br_query).show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2022|    292725.69|
|2021|    294211.46|
|2020|    294204.16|
|2019|    287287.82|
+----+-------------+



In [8]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_2flrs_query = """
  SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS Average_Price
  FROM homes
  WHERE bedrooms=3
  AND bathrooms=3
  AND floors=2
  AND sqft_living>=2000
  GROUP BY Year
  ORDER BY Year DESC
"""
spark.sql(avg_2flrs_query).show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2022|    290242.99|
|2021|    296330.96|
|2020|    292289.09|
|2019|    289859.14|
+----+-------------+



In [9]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
view_query = """
  SELECT view AS View_Rating, ROUND(AVG(price), 2) AS Average_Price
  FROM homes
  GROUP BY View_Rating
  HAVING Average_Price >= 350000
  ORDER BY View_Rating DESC
"""

In [10]:
# Although this is a small dataset, determine the run time for this query.
print("Is table cached?", spark.catalog.isCached('homes'))
start_time = time.time()
spark.sql(view_query).show()
print("--- %s seconds ---" % (time.time() - start_time))

Is table cached? False
+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|        100|    1026669.5|
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|    1054325.6|
|         94|    1033536.2|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|    1072285.2|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|    1063498.0|
|         81|   1053472.79|
+-----------+-------------+
only showing top 20 rows

--- 0.9549717903137207 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
print("Is table cached?", spark.catalog.isCached('homes'))

spark.catalog.cacheTable('homes')

print("Is table cached?", spark.catalog.isCached('homes'))

Is table cached? False
Is table cached? True


In [12]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
print("Is table cached?", spark.catalog.isCached('homes'))

start_time = time.time()

# Just rerun the query and not rewrite it
spark.sql(view_query).show()
print("--- %s seconds ---" % (time.time() - start_time))

Is table cached? True
+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|        100|    1026669.5|
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|    1054325.6|
|         94|    1033536.2|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|    1072285.2|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|    1063498.0|
|         81|   1053472.79|
+-----------+-------------+
only showing top 20 rows

--- 1.9538230895996094 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homes_df.write.partitionBy("date_built").mode("overwrite").parquet("homes_partitioned")

In [14]:
# 11. Read the parquet formatted data.
homes_df_part=spark.read.parquet('homes_partitioned')

In [15]:
# 12. Create a temporary table for the parquet data.
homes_df_part.createOrReplaceTempView('homes_part')

In [16]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

view_query = """
  SELECT view AS View_Rating, ROUND(AVG(price), 2) AS Average_Price
  FROM homes_part
  GROUP BY View_Rating
  HAVING Average_Price >= 350000
  ORDER BY View_Rating DESC
"""

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00011610984802246094 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
print("Is table cached?", spark.catalog.isCached('homes'))

spark.catalog.uncacheTable('homes')

print("Is table cached?", spark.catalog.isCached('homes'))

Is table cached? True
Is table cached? False
